In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
Pkg.add("Statistics")
Pkg.add("Plots")
using Plots
using Statistics

  Activating project at `c:\Users\13158\Downloads`
   Resolving package versions...
  No Changes to `C:\Users\13158\Downloads\Project.toml`
  No Changes to `C:\Users\13158\Downloads\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\13158\Downloads\Project.toml`
  No Changes to `C:\Users\13158\Downloads\Manifest.toml`


In [ ]:

#Define parameters of the design
area = 13950 #square feet
phos_limit = 40 #lbs/day
bod_limit = 2502 #lbs/day
flow_rate = 6.5/4 #MGD Four tanks
flow_liter = flow_rate*4404883.8
phos_influent = 7 #mg/liter
bod_influent = 300 #mg/liter
tank_depth = 12 #ft
tank_volume = area*tank_depth #cubic feet
volume_mg = tank_volume * 7.48/4 #million gallons
Y =.5 #Bacteria synthesis yield (standard value)
b = .08 #Endogenous decay coefficient (standard value)


#Compute an estimate for the maximum SRT allowable for 
#The area given for the tanks
was_rate = volume_mg/39.5 #gallons/day
was_conc = 4000 # mg/liter
mlss = 3500*8.34 #mg/liter
max_srt = round(volume_mg/((was_conc*was_rate*8.34)/mlss))
max_srt = Int(max_srt)

#Store the average and maximum phosphorus and BOD concentration
#For each SRT to determine efficacy of treatment at that SRT
max_phos = zeros(max_srt)
max_bod = zeros(max_srt)
avg_phos = zeros(max_srt)
avg_bod = zeros(max_srt)

#Test system over a variety of different 
#phosphorus and BOD values
phos_influent = collect(1:15) #mg/L
bod_influent = collect(50:50:750) #mg/L
srt_array = 1:max_srt

#Write a function to compute changes in phosphorus and BOD concentrations
#Within the system and return effluent values
function conc_change(Y, b, srt, phos_influent, bod_influent, flow_liter)
    #Convert BOD to readily biodegradable COD
    rbcod = (bod_influent./2).*.2
    #Compute phosphorus removal in aeration zone based on PAO
    #Growth in anaerobic zone
    phos_new = phos_influent .- ((Y./ (1 .+ b.* srt)).* (rbcod.* 0.9).*.4)
    #Compute BOD removal in aerobic zone

    S = (8*(1+.088*srt))/(srt*(3.5-.088)-1)
    So = 1.6 .*bod_influent
    bod_new = bod_influent .- (So.-S)./(1 .+ .088 .*srt)
    
    rbcod_new = (bod_new./2).*.2
    #Repeat process for second aeration zone to get effluent concentration values
    phos_effluent = phos_new .- ((Y./(1 .+b.*srt).*(rbcod_new.*.9))*.4) 
    
    So = 1.6 .* bod_new
    bod_effluent =  3 .+ bod_new .- (So .-S)./(1 .+ .088 .*srt)
    

    return phos_effluent, bod_effluent

end

count = 1
#Run the function for each SRT value
while count <= max_srt -1
    srt = srt_array[count]
    phos_effluent, bod_effluent = conc_change(Y, b, srt, phos_influent, bod_influent, flow_liter)
    max_phos[count] = maximum(phos_effluent)
    max_bod[count] = maximum(bod_effluent)
    avg_phos[count] = mean(phos_effluent)
    avg_bod[count] = mean(bod_effluent)
    count = count + 1
end



#Convert concentrations from mg/L to lbs/day
maxphos_lbs = max_phos .* 24605176.596 .* .0000002204623
maxbod_lbs = max_bod .* 24605176.596 .* .0000002204623
avgphos_lbs = avg_phos .* 24605176.596 .* .0000002204623
avgbod_lbs = avg_bod .* 24605176.596 .* .0000002204623

#Create a plot
p = scatter(srt_array,maxbod_lbs, legend = false)
ylabel!("Max BOD discharged to lake (lbs/day)")
xlabel!("SRT (days)")

#Print values for the optimal SRT
print(avgbod_lbs[7])
print(avgphos_lbs[7])
print(maxphos_lbs[7])
print(maxbod_lbs[7])


18.40137403565402318.0901286202844833.9381370207891418.587491067169672

In [1]:
test = 1:34
print(test)
print(test[5])

1:345